# Informe: Identificación de Modelo FOPDT utilizando el Método de Smith

## Introducción
En este informe presentamos el proceso de identificación de un modelo FOPDT (First Order Plus Dead Time) para el sistema térmico TempControlLab UdeA, utilizando el método de Smith. El objetivo es caracterizar la dinámica del sistema a partir de datos experimentales obtenidos mediante una prueba de escalón en lazo abierto. El análisis se realizó utilizando Python, con bibliotecas como `pandas`, `numpy` y `matplotlib` para el procesamiento y visualización de datos.

## 1. Lectura y preparación de los datos
**Objetivo**: Cargar los datos experimentales almacenados en un archivo de texto y extraer las variables relevantes para el análisis.

En este apartado:
- Utilizamos `pandas` para leer el archivo `Datos.txt`, que contiene las mediciones de tiempo, temperatura (T1) y señal PWM aplicada al calentador.
- Las columnas se extraen como arreglos de `numpy` para facilitar el procesamiento numérico.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Leer los datos (asegúrate de que Datos.txt esté en la misma carpeta)
# df = pd.read_csv('Datos.txt')  # Descomentar para uso real
# 2. Extraer columnas
# tiempo = df['Tiempo (s)'].values
# T1 = df[' T1 (C)'].values
# pwm = df[' PWM_Heater1'].values

###################################
# PARA EJEMPLO SIN DATOS REALES   #
###################################

# Para fines demostrativos, crearemos datos sintéticos:
tiempo = np.linspace(0, 300, 301)
# Suponiendo que la señal PWM pasa de 0 a 50 en t=50s.
pwm = np.zeros_like(tiempo)
pwm[tiempo >= 50] = 50

# Simulamos una respuesta térmica sencilla:
# (Sólo a modo ilustrativo, NO son datos reales)
T1 = 25 + (pwm/50)* (1 - np.exp(-(tiempo - 50)/60))
T1[tiempo < 50] = 25  # Temperatura constante antes del escalón

print("Datos simulados cargados.\n")
print("Primeros 10 valores de tiempo:", tiempo[:10])
print("Primeros 10 valores de T1:", T1[:10])
print("Primeros 10 valores de pwm:", pwm[:10])

## 2. Detección del escalón de entrada
**Objetivo**: Identificar el instante en el que se aplica el escalón de potencia al sistema, lo que marca el inicio de la respuesta transitoria.

En este apartado:
- Se busca el punto donde la señal PWM cambia de 0 a un valor positivo.
- Si no se detecta, el programa se detiene. De lo contrario, se imprime el tiempo en el que ocurre el cambio.

In [ ]:
# 3. Detectar el instante donde se aplica el escalón
indices_escalon = np.where((pwm[:-1] == 0) & (pwm[1:] > 0))[0]
if len(indices_escalon) == 0:
    print("No se detectó un escalón.")
    # exit() # Descomentar si se desea finalizar la ejecución
else:
    idx_escalon = indices_escalon[0]
    t_escalon = tiempo[idx_escalon + 1]
    print(f"Escalón detectado en t = {t_escalon:.2f} s")

## 3. Recorte de datos y determinación de valores inicial y final
**Objetivo**: Aislar la respuesta del sistema a partir del instante del escalón y determinar las temperaturas inicial y final para normalizar los datos.

En este apartado:
- Se recortan los datos para considerar solo el período posterior al escalón.
- `y0` y `yf` representan la temperatura inicial (antes del escalón) y final (en estado estacionario), respectivamente. `delta_y` es el cambio total en la temperatura.

In [ ]:
# 4. Recortar datos desde el escalón en adelante
tiempo = tiempo[idx_escalon:]
T1 = T1[idx_escalon:]
pwm = pwm[idx_escalon:]

# 5. Determinar valores inicial y final
y0 = T1[0]
yf = T1[-1]
delta_y = yf - y0
print(f"Temperatura inicial (y0): {y0:.2f} °C")
print(f"Temperatura final (yf): {yf:.2f} °C")
print(f"Cambio total (delta_y): {delta_y:.2f} °C")

## 4. Cálculo de puntos característicos (28.3% y 63.2%)
**Objetivo**: Localizar los tiempos en los que la respuesta alcanza el 28.3% y 63.2% del cambio total, puntos clave para el método de Smith.

En este apartado:
- Se calculan los valores de temperatura correspondientes a esos porcentajes.
- Se buscan los instantes de tiempo en que la temperatura se acerca más a esos valores.

In [ ]:
# 6. Calcular valores objetivo al 28.3% y 63.2% del cambio
y_28 = y0 + 0.283 * delta_y
y_63 = y0 + 0.632 * delta_y

# 7. Encontrar tiempos donde se alcanza ese valor
t_28 = tiempo[np.argmin(np.abs(T1 - y_28))]
t_63 = tiempo[np.argmin(np.abs(T1 - y_63))]

print(f"Tiempo para 28.3%: {t_28:.2f} s")
print(f"Tiempo para 63.2%: {t_63:.2f} s")

## 5. Estimación de parámetros del modelo FOPDT
**Objetivo**: Calcular los parámetros del modelo (ganancia $K$, constante de tiempo $\tau$ y tiempo muerto $L$) utilizando las fórmulas del método de Smith.

En este apartado:
- $\tau$ se calcula como 1.5 veces la diferencia entre los tiempos al 63.2% y 28.3%.
- $L$ se obtiene restando $\tau$ de $t_{63}$.
- $K$ es la ganancia del sistema, calculada como el cambio en la temperatura dividido por el escalón en la señal PWM.

In [ ]:
# 8. Calcular constantes del modelo
tau = 1.5 * (t_63 - t_28)
L = t_63 - tau

# Asumiendo que el escalón fue de 0 a pwm[idx_escalon + 1]
# (luego de haber recortado, idx_escalon es 0 en este subset, por tanto pwm[0])
K = delta_y / pwm[0] if pwm[0] != 0 else 0

print("Modelo aproximado:")
print(f"  Ganancia (K) = {K:.3f}")
print(f"  Constante de tiempo (tau) = {tau:.2f} s")
print(f"  Retardo (L) = {L:.2f} s")

## 6. Simulación del modelo FOPDT
**Objetivo**: Generar la respuesta del modelo estimado para compararla con los datos reales.

En este apartado:
- Para tiempos menores que $L$, la respuesta del modelo es igual a la temperatura inicial ($y_0$).
- Para tiempos mayores, se aplica la ecuación de un sistema de primer orden con retardo.

In [ ]:
# 9. Simular el modelo de primer orden con retardo
t_modelo = tiempo.copy()
respuesta_modelo = np.zeros_like(t_modelo)

for i, t in enumerate(t_modelo):
    if t < L:
        respuesta_modelo[i] = y0
    else:
        respuesta_modelo[i] = y0 + K * pwm[0] * (1 - np.exp(-(t - L) / tau))

print("Simulación del modelo completada.")

## 7. Visualización de resultados
**Objetivo**: Graficar los datos reales y la respuesta del modelo para evaluar la calidad del ajuste.

En este apartado:
- Se muestran los datos reales (línea continua) y la respuesta del modelo (línea discontinua).
- Se incluye una línea vertical que indica el tiempo muerto $L$.

In [ ]:
# 10. Graficar comparación
plt.figure()
plt.plot(tiempo, T1, label='Datos reales')
plt.plot(t_modelo, respuesta_modelo, '--', label='Modelo de Smith')
plt.axvline(L, linestyle='--', label='Tiempo muerto')
plt.xlabel('Tiempo [s]')
plt.ylabel('Temperatura [°C]')
plt.title('Identificación con Método de Smith')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## Resultados
Los parámetros estimados del modelo FOPDT fueron:
- **Ganancia (K)**: *(valor calculado impreso arriba)*
- **Constante de tiempo (tau)**: *(valor calculado impreso arriba)*
- **Retardo (L)**: *(valor calculado impreso arriba)*

La gráfica resultante muestra un buen ajuste entre los datos reales y el modelo, especialmente en la región de estado estacionario. Sin embargo, se observan pequeñas discrepancias en la fase transitoria, lo que podría deberse a efectos no lineales no considerados en el modelo FOPDT.

## Conclusiones
El método de Smith proporcionó una estimación rápida y efectiva de los parámetros del modelo FOPDT utilizando puntos clave de la respuesta al escalón, capturando la dinámica general del sistema, aunque las discrepancias en la fase transitoria sugieren que podrían explorarse modelos más complejos para mejorar la precisión.

El código desarrollado demostró ser robusto y reutilizable para análisis de otros conjuntos de datos o sistemas similares. Para validar los resultados, sería recomendable contrastarlos con otros métodos como Ziegler-Nichols o herramientas de identificación de MATLAB, lo que permitiría verificar la consistencia de los parámetros estimados.

Este trabajo evidencia la utilidad del método de Smith en la identificación de sistemas térmicos, sentando una base sólida para el diseño de estrategias de control avanzadas.